<h1 align="center">[Fine Tune] Fine Tuning BERT for Question Answering (QA) Task</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro 

`Question Answering` (QA) é um tipo de tarefa de processamento de linguagem natural em que um modelo é treinado para responder a perguntas com base em um determinado contexto ou passagem de texto. O modelo `BERT` do Google é uma rede neural baseada em `Transformers` pré-treinada que se mostrou eficaz em muitas tarefas de processamento de linguagem natural, incluindo controle de qualidade. Para melhorar o desempenho do `BERT` em uma tarefa de controle de qualidade, o modelo pode ser ajustado em um conjunto de dados menor específico para essa tarefa. Isso permite que o modelo aprenda recursos específicos da tarefa e melhore seu desempenho.

# Dataset

In [1]:
#!nvidia-smi

# Verificamos CUDA com o Pytorch:
import torch

torch.cuda.is_available()

True

In [ ]:
# Versão leve:
#!pip install transformers

# Versão de Desenvolvimento:
#%pip install transformers[sentencepiece]
# !pip install transformers

%pip install datasets evaluate transformers[sentencepiece]

In [18]:
from huggingface_hub import notebook_login 

notebook_login() 

In [19]:
# Adicionamos nosso e-mail, também, para que se vaja COMMITANDO automaticamente todo o progresso do Treinamento

!git config --global user.email "eddychirinos.unac@gmail.com"

In [20]:
# Devido ao tamanho do que podem ocupar estes modelos

%pip install git-lfs

In [21]:
import transformers

print(transformers.__version__)

4.26.1


In [ ]:
import pandas as pd

from datasets import load_dataset 

squad = load_dataset( "squad" , split= "train[:5000]" ) 
squad = squad.train_test_split(test_size= 0.2 ) 


In [23]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

Para poder usar nosso conjunto de dados personalizado no futuro, convertemos o conjunto de dados do formato de `hub HuggingFace` em um `Pandas DataFrame tradicional`. Isso nos ajudará a entender como importar nosso conjunto de dados personalizado no futuro.

In [25]:
# Converte o conjunto de dados em um dicionário
data_dict = squad[ "train" ].to_dict()

# Cria um DataFrame do dicionário
df = pd.DataFrame.from_dict(data_dict)
df.head()

,id,title,context,question,answers
0,56bec29b3aeaaa14008c9380,Beyoncé,"In 2006, Beyoncé introduced her all-female tou...",Beyonce had singers in the background known by...,"{'text': ['The Mamas'], 'answer_start': [216]}"
1,56cfde92234ae51400d9bfb7,Frédéric_Chopin,"Although this period had been productive, the ...",Who did Sand sell Chopin's piano to?,"{'text': ['the Canuts'], 'answer_start': [228]}"
2,56cd4a5162d2951400fa6517,Sino-Tibetan_relations_during_the_Ming_dynasty,"According to Tibetologist John Powers, Tibetan...",Who did the Ming emperors send invitations to?,"{'text': ['ruling lamas'], 'answer_start': [416]}"
3,56ce345caab44d1400b88581,New_York_City,The first documented visit by a European was i...,What was the name of the first European who ar...,"{'text': ['Giovanni da Verrazzano'], 'answer_s..."
4,56beb2a43aeaaa14008c923a,Beyoncé,"Following the death of Freddie Gray, Beyoncé a...",Beyonce with Jay Z gave lots of money to bail ...,"{'text': ['protesters'], 'answer_start': [132]}"


In [26]:
df.shape

(4000, 5)

# Pré-processamento no Dataset de Treinamento

Este código, abaixo, está fazendo o pré-processamento em um conjunto de dados armazenado em um DataFrame chamado `“df”`. `Primeiro`, ele está usando o `AutoTokenizer` da biblioteca `Transformers` para tokenizar as perguntas e o contexto do DataFrame. Ele está passando os parâmetros `questions`, `context`, `max_length` e outros para a função tokenizadora. A função tokenizer retorna um dicionário contendo a entrada tokenizada, que é armazenada na variável inputs. O código também recupera o `offset_mapping` das entradas, que é usado para calcular as posições inicial e final das respostas no contexto. Essas posições são armazenadas nas listas `start_positions` e `end_positions`, que são adicionadas como colunas ao DataFrame.

Depois disso, ele cria um novo DataFrame a partir dos dados de entrada e das listas `start_positions` e `end_positions`, em seguida, salva o DataFrame em um arquivo chamado CSV `'encoding_train.csv'`. Ele também está convertendo o DataFrame em um Pandas DataFrame tradicional e carregando-o como um conjunto de dados de Train. Esta etapa de pré-processamento está preparando os dados de entrada e as posições de resposta para treinar um modelo de resposta a perguntas. Ao tokenizar a entrada, é possível que o modelo processe a entrada e, ao calcular as posições inicial e final das respostas no contexto, é possível que o modelo aprenda a encontrar as respostas na entrada.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answers']
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

import pandas as pd
from datasets import Dataset
data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_train.csv',index=False)
train = Dataset.from_pandas(df)

# Pré-processamento no Dataset de Teste

In [30]:
import pandas as pd 
# Convert the dataset to a dictionary
data_dict = squad["test"].to_dict()
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

questions = [q.strip() for q in df["question"]]
context = [q.strip() for q in df["context"]]
inputs = tokenizer(
        questions,
        context,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

offset_mapping = inputs.pop("offset_mapping")

start_positions = []
end_positions = []
answers = df['answers']
for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

df["start_positions"] = start_positions
df["end_positions"] = end_positions

data = {'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'start_positions':start_positions,
        'end_positions': end_positions,
       }
df = pd.DataFrame(data)
df.to_csv('encoding_test.csv',index=False)
test = Dataset.from_pandas(df)

# Ajustando o BERT a nosso conjunto de Dados

Repara que estamos usando o modelo `distilbert`:

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import DefaultDataCollator


model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



In [ ]:
# Agora, Treinamos:

trainer.train()

Neste script, orientamos você pelo processo de ajuste fino do BERT para tarefas de controle de qualidade . O ajuste fino do BERT em uma tarefa de controle de qualidade é benéfico porque requer menos dados de treinamento em comparação com o treinamento de um modelo do zero e pode ser facilmente adaptado para novas tarefas de controle de qualidade.